<div class="alert alert-block alert-info">
    <b>Import Packages</b>
</div>


In [ ]:
import pandas as  pd
import os
import numpy as np
import dtale
import json
from tqdm import tqdm
import sys

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [ ]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>View .csv files in our working directory</b>
</div>


In [ ]:
for file in os.listdir():
    if file.endswith(".csv"):
        print(os.path.join("/", file))

<div class="alert alert-block alert-info">
    <b>Create df by import .csv files created in previous step</b>
</div>


In [ ]:
df_patients = pd.read_csv('patients.csv')#.iloc[:1000,:]
df_episodes = pd.read_csv('df_episode.csv')#.iloc[:150,:]
df_visits = pd.read_csv('df_visits.csv')#.iloc[:150,:]
df_OASIS = pd.read_csv('df_OASIS.csv')#.iloc[:150,:]
df_event_mapping = pd.read_csv('event_mapping_df.csv')

<div class="alert alert-block alert-info">
    <b>This allows us to filter our data to only a handful of patients for testing purposes</b>
</div>


In [ ]:
alive = df_patients[df_patients.mortality==0].pa_id

In [ ]:
alive.dtype
len(alive)

In [ ]:
dead = df_patients[df_patients.mortality == 1].pa_id

In [ ]:
dead.dtype
len(dead)

In [ ]:
df_OASIS_dead = df_OASIS[df_OASIS.pa_id.isin(dead)]
df_OASIS_dead.dtypes

In [ ]:
grpz = df_OASIS_dead.groupby('pa_id').groups
#df_OASIS.groupby('pa_id').groups[36796]
#df_OASIS.groupby('pa_id').get_group(tup)

In [ ]:
len(grpz)

In [ ]:
def patient_most_rows(grpz):
    maxcount = max(len(v) for v in grpz.values())
    print(f'The max rownum is {maxcount}, and the pa_id is:')
    return [k for k, v in grpz.items() if len(v) == maxcount]


In [ ]:
patient_most_rows(grpz)

In [ ]:
length_dict = {k: len(v) for k, v in grpz.items()}

In [ ]:
[k for k, v in length_dict.items() if v == max(length_dict.values())]
import heapq
#[k for k, v in length_dict.items() if v in heapq.nlargest(100, length_dict.values())]
dead100 = [k for k, v in length_dict.items() if v in heapq.nlargest(100, length_dict.values())]
len(dead100)

In [ ]:
pa_subset = alive.tolist() + dead100
len(set(pa_subset))

In [ ]:




#grpidx = (17191,44654,25607,18808,16862,17018,24556,16274,26475,27968,36781,17747,37870,18011,30512,51094,38501,36509,17716,42124,16950,353347,326385,319247,318537,307867,283257,268061,347156,325432,348164,348170,350202,335984,359871,364367,320405,306152,322487,322263,308977,280963)
grpidx = pa_subset
#dfidx = np.sort(np.concatenate([df_OASIS.groupby('pa_id').indices[x] for x in grpidx]))
#df_OASIS.loc[dfidx, :]


<div class="alert alert-block alert-info">
    <b>Repopulate the original df with test data</b>
</div>


In [ ]:
df_patients = df_patients[df_patients.pa_id.isin(grpidx)]
df_episodes = df_episodes[df_episodes.pa_id.isin(grpidx)]
df_visits = df_visits[df_visits.pa_id.isin(grpidx)]
df_OASIS = df_OASIS[df_OASIS.pa_id.isin(grpidx)]

In [ ]:
df_OASIS.pa_id.nunique()

<div class="alert alert-danger">
    <b>View datatypes - the int64 were causing a bug when serializing to JSON</b>
</div>

In [ ]:
df_patients.dtypes#.head()
df_episodes.dtypes#.head()
df_visits.dtypes#.head()
df_OASIS.dtypes#.head()
df_event_mapping.dtypes#.head()

<div class="alert alert-warning">
    <b>Import packages needed to Parallel process the Data</b>
</div>

In [ ]:
import joblib
from joblib import Parallel, delayed

<div class="alert alert-warning">
    <b>View the number of cores recognized for parallel processing</b>
</div>

In [ ]:

print(joblib.cpu_count() - 4)


<div class="alert alert-warning">
    <b>Import pyhealth classes and functions</b>
</div>

In [ ]:
from pyhealth.data.base_mimic import parallel_parse_tables
from pyhealth.utils.utility_parallel import unfold_parallel
from pyhealth.utils.utility_parallel import partition_estimators
from pyhealth.utils.utility import read_csv_to_df
from pyhealth.utils.utility import make_dirs_if_not_exists

<div class="alert alert-warning">
    <b>Create Output Directory to Save Files</b>
</div>

In [ ]:
#from pathlib import Path
#Path("pyhealth_export").mkdir(parents=True, exist_ok=True)

<div class="alert alert-warning">
    <b>Setup the Parallel Proccesing Workflow</b>
</div>

In [ ]:
save_dir = './pyhealth_export3'
n_jobz = (joblib.cpu_count() - 4)  # number of parallel jobs
duration = 21600  # time window for episode generation
selection_method = 'last'
var_list = df_event_mapping.OASIS.tolist()
patient_id_list = df_patients['pa_id'].tolist()

In [ ]:
df_event_mapping.iloc[-1]['OASIS']
df_event_mapping

<div class="alert alert-danger">
    <b>Custom JSON Encoder to deal with np.int64 serialization Bug</b>
</div>

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

<div class="alert alert-warning">
    <b>Execute the Data Processing</b>
</div>

In [ ]:
n_patients_list, starts, n_jobs = partition_estimators(
        len(patient_id_list), n_jobs=n_jobz)

In [ ]:
#patient_id_list
n_patients_list # you can see the batch being separates
starts # cummulative sum of number of jobs
n_jobs

In [ ]:
from pyhealth.data.base_mimic import parallel_parse_tables
all_results = Parallel(n_jobs=n_jobs, max_nbytes=None, verbose=True)(
delayed(parallel_parse_tables)(
     patient_df=df_patients,
     admission_df=df_episodes,
      icu_df=df_visits,
     event_df=df_OASIS,
     event_mapping_df=df_event_mapping,
     duration=duration,
     save_dir=save_dir,selection_method = selection_method, var_list = var_list, patient_id_list = patient_id_list)
 for i in range(n_jobs))

In [ ]:
patient_data_loc = os.path.join(save_dir, 'patient_data_demo.json')
all_results = list(map(list, zip(*all_results)))
valid_data_list = unfold_parallel(all_results[0], n_jobs)
valid_id_list = unfold_parallel(all_results[1], n_jobs)

patient_data_list = []
for p in valid_data_list:
        patient_data_list.append(p.data)

with open(patient_data_loc, 'w') as outfile:
        json.dump(patient_data_list, outfile, cls=NpEncoder)



<div class="alert alert-danger">
    <b>Examination of the Output</b>
</div>

In [ ]:
#patient_data_loc
#all_results
#type(valid_data_list[2])   # WHY ARE THESE INTEGERS IN THIS LIST
#valid_data_list[0].data
#valid_id_list[0].data
#patient_data_list[0]
valid_id_list
valid_data_list


In [ ]:
type(all_results[0])

In [ ]:
type(all_results[0][0][0])

In [ ]:
all_results[0][0][0]

In [ ]:
print(all_results[0][0][0])

In [ ]:
type(all_results[0][0][0].data['dob'])
#all_results[0][0][0].data

In [ ]:
all_results[0][0][0].data

In [ ]:
patient_data_list

<div class="alert alert-warning">
    <b>Generate Labels for Mortality or Not</b>
</div>

In [ ]:
    with open(patient_data_loc) as f:
        patient_data_list = json.load(f)

    ##########################################################
    output_headers = ['episode_file', 'death_indicator']

    output_df = pd.DataFrame(columns=output_headers)

    # for i, p_id in tqdm(enumerate(patient_data_list), total=len(patient_data_list)):
    for i, p_id in enumerate(tqdm(patient_data_list)):
        for adm in p_id['admission_list']:
            # csv file does not exist
            if 'episode_csv' in adm.keys():
                temp_list = [adm['episode_csv'], adm['death_indicator']]

                # append to the major episode dataframe
                temp_df = pd.DataFrame(temp_list).transpose()
                temp_df.columns = output_headers
                output_df = pd.concat([output_df, temp_df], axis=0)

    # change file header to lower case
    output_df.columns = output_df.columns.str.lower()
    output_df.to_csv(
        os.path.join(save_dir, 'y_mortality_mimic_demo.csv'),
        index=False)

In [ ]:
root_dir= os.getcwd()

In [ ]:
root_dir
data_dir = os.path.join(root_dir, 'pyhealth_export3')
data_dir

In [ ]:
from __future__ import division
from __future__ import print_function
from pathlib import Path


In [ ]:
from pyhealth.data.expdata_generator import sequencedata as expdata_generator
from pyhealth.evaluation.evaluator import func

In [ ]:
### May choose any of these models
# from pyhealth.models.sequence.dipole import Dipole as model
from pyhealth.models.sequence.lstm import LSTM as model
#from pyhealth.models.sequence.gru import GRU as GRU
# from pyhealth.models.sequence.embedgru import EmbedGRU as model
# from pyhealth.models.sequence.retain import Retain as model
# from pyhealth.models.sequence.raim import RAIM as model
# from pyhealth.models.sequence.tlstm import tLSTM as model
# from pyhealth.models.sequence.xgboost import XGBoostECG as model
# from pyhealth.models.sequence.rf import RandomForest as model


In [ ]:
data_dir = os.path.join(root_dir, 'pyhealth_export')
expdata_id = '2020.0810.data.mortality.mimic'

    # set up the datasets
cur_dataset = expdata_generator(expdata_id, root_dir=root_dir)
cur_dataset.get_exp_data(sel_task='mortality', data_root=data_dir)
cur_dataset.load_exp_data()
cur_dataset.show_data()


In [ ]:
import torch

In [ ]:
print(torch.__version__)

In [ ]:




    # initialize the model for training
    # turn on GPU by setting use_gpu to True
expmodel_id = '2020.0810.gru.data.mortality.mimic.gpu'
clf = model(expmodel_id=expmodel_id, n_batchsize=20, use_gpu=False,
                n_epoch=50, gpu_ids='0, 1')
clf.fit(cur_dataset.train, cur_dataset.valid)

    # load the best model for inference
clf.load_model()
clf.inference(cur_dataset.test)
pred_results = clf.get_results()
print(pred_results['hat_y'])

# evaluate the model
r = func(pred_results['hat_y'], pred_results['y'])
print(r)

In [ ]:
cur_dataset.train
cur_dataset.valid
cur_dataset.

In [ ]:
clf.inference(cur_dataset.valid)

In [ ]:
pred_results

In [ ]:
r